In [118]:
# soo jiido import liabrarys
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [119]:
# load dataset, check HEAD,INFO,describe,MISSING VALUES
df = pd.read_csv("customers_l9_dataset.csv")
print(df.head())
print(df.info())
print(df.describe())
print(df.isnull().sum())

   CustomerID  Gender  Age  Annual Income ($)  Spending Score (1-100)
0           1    Male   19                 15                      39
1           2    Male   21                 15                      81
2           3  Female   20                 16                       6
3           4  Female   23                 16                      77
4           5  Female   31                 17                      40
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CustomerID              200 non-null    int64 
 1   Gender                  200 non-null    object
 2   Age                     200 non-null    int64 
 3   Annual Income ($)       200 non-null    int64 
 4   Spending Score (1-100)  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB
None
       CustomerID         Age  Annual Income ($)  Spendin

In [120]:
#Prepare Features 
#get feutures
FEATURES = ["Annual Income ($)", "Spending Score (1-100)"]
x = df[FEATURES].copy()

In [121]:
for col in FEATURES:
  if x[col].isna().any():
    x[col] = x[col].fillna(x[col].median())

In [122]:
#scaller  yuusan model moodin in ay kala muhiimsan yihiin
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
print(f"after scaled", x_scaled)

after scaled [[-1.73899919 -0.43480148]
 [-1.73899919  1.19570407]
 [-1.70082976 -1.71591298]
 [-1.70082976  1.04041783]
 [-1.66266033 -0.39597992]
 [-1.66266033  1.00159627]
 [-1.62449091 -1.71591298]
 [-1.62449091  1.70038436]
 [-1.58632148 -1.83237767]
 [-1.58632148  0.84631002]
 [-1.58632148 -1.4053405 ]
 [-1.58632148  1.89449216]
 [-1.54815205 -1.36651894]
 [-1.54815205  1.04041783]
 [-1.54815205 -1.44416206]
 [-1.54815205  1.11806095]
 [-1.50998262 -0.59008772]
 [-1.50998262  0.61338066]
 [-1.43364376 -0.82301709]
 [-1.43364376  1.8556706 ]
 [-1.39547433 -0.59008772]
 [-1.39547433  0.88513158]
 [-1.3573049  -1.75473454]
 [-1.3573049   0.88513158]
 [-1.24279661 -1.4053405 ]
 [-1.24279661  1.23452563]
 [-1.24279661 -0.7065524 ]
 [-1.24279661  0.41927286]
 [-1.20462718 -0.74537397]
 [-1.20462718  1.42863343]
 [-1.16645776 -1.7935561 ]
 [-1.16645776  0.88513158]
 [-1.05194947 -1.7935561 ]
 [-1.05194947  1.62274124]
 [-1.05194947 -1.4053405 ]
 [-1.05194947  1.19570407]
 [-1.01378004 -

In [123]:
#Elbow method   
# soo hel k da
for k in range(1, 12):
  print("k", k)
  km = KMeans(n_clusters=k, random_state=42) 
  km.fit(x_scaled)
  print(f"K = {k} ==> SSE = {km.inertia_:.2f}")

k 1
K = 1 ==> SSE = 400.00
k 2
K = 2 ==> SSE = 273.67
k 3
K = 3 ==> SSE = 157.70
k 4
K = 4 ==> SSE = 109.23
k 5
K = 5 ==> SSE = 65.57
k 6
K = 6 ==> SSE = 60.13
k 7
K = 7 ==> SSE = 49.67
k 8
K = 8 ==> SSE = 37.32
k 9
K = 9 ==> SSE = 32.50
k 10
K = 10 ==> SSE = 30.06
k 11
K = 11 ==> SSE = 26.76


In [124]:
#Evaluate Clustering
SIL = silhouette_score(x_scaled, Labels)
DBI = davies_bouldin_score(x_scaled, Labels)

In [125]:
#wan helay k da
kmeans = KMeans(n_clusters=6, random_state=42, n_init=10)
Labels = kmeans.fit_predict(x_scaled)

df["Cluster_Groups"] = Labels.astype(int)
print("Cluster sample")
print(df.head())

Cluster sample
   CustomerID  Gender  Age  Annual Income ($)  Spending Score (1-100)  \
0           1    Male   19                 15                      39   
1           2    Male   21                 15                      81   
2           3  Female   20                 16                       6   
3           4  Female   23                 16                      77   
4           5  Female   31                 17                      40   

   Cluster_Groups  
0               3  
1               4  
2               3  
3               4  
4               3  


In [126]:
#metrics
print("__Metrics__")
print(f"silhouette_score: {SIL:.3f} (closer +1 is good) ") 
print(f"davies_bouldin_score: {DBI:.3f} (lower is better) ") 

__Metrics__
silhouette_score: 0.540 (closer +1 is good) 
davies_bouldin_score: 0.655 (lower is better) 


In [127]:
#Soo celinta centers, Cluster Centers Original Units
centers_scaled = kmeans.cluster_centers_
centers_original = scaler.inverse_transform(centers_scaled) 


#update dataframe 
centers_df = pd.DataFrame(centers_original, columns=FEATURES)
print("changed to original unit: ", centers_df.round(2))

changed to original unit:     Annual Income ($)  Spending Score (1-100)
0              55.30                   49.52
1              88.20                   17.11
2             109.70                   82.00
3              26.30                   20.91
4              25.73                   79.36
5              78.55                   82.17


In [128]:
#sanit check
sample_idx = [0, 1, 2, 3, 4 ]
sanity = df.loc[sample_idx, FEATURES + ["Cluster_Groups"]]
print("sanity_check (5 customers)")
print(sanity)


sanity_check (5 customers)
   Annual Income ($)  Spending Score (1-100)  Cluster_Groups
0                 15                      39               3
1                 15                      81               4
2                 16                       6               3
3                 16                      77               4
4                 17                      40               3


In [129]:
#save gareyn
OUT_PATH = "spending_labeled_clusters.csv"
df.to_csv(OUT_PATH, index=False)
print("saved Dataset   Name:",OUT_PATH)

saved Dataset   Name: spending_labeled_clusters.csv
